In [1]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import os
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import regularizers
from keras.regularizers import l1
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
import time

Using TensorFlow backend.


In [2]:
num_classes = 6
img_rows, img_cols = 48, 48
batch_size = 64
train_data_dir = './Dataset/Train_dataset'
validation_data_dir = './Dataset/Validation_dataset'

In [3]:
# Let's use some data augmentaiton 
# train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(
        rescale=1./255,
      rotation_range=35,
      shear_range=0.3,
      zoom_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

Found 1200 images belonging to 6 classes.
Found 249 images belonging to 6 classes.


C:\ProgramData\Anaconda3\lib\site-packages\keras_preprocessing\image\utils.py:156: UserWarning: Using ".tiff" files with multiple bands will cause distortion. Please verify your output.
  warnings.warn('Using ".tiff" files with multiple bands '


In [4]:
print(validation_generator.class_indices)

{'anger': 0, 'disgust': 1, 'happiness': 2, 'neutral': 3, 'sadness': 4, 'surprise': 5}


In [5]:
# Create the model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',kernel_regularizer=regularizers.l2(0.0001),input_shape=(48,48,1)))
# model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu',kernel_regularizer=regularizers.l2(0.0001)))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(6, kernel_size=(1, 1), activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
# # model.add(BatchNormalization())

model.add(Conv2D(6, kernel_size=(4, 4), activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
# model.add(BatchNormalization())

model.add(Flatten())

model.add(Activation("softmax"))

model.summary()
# model.add(Dense(1024, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(7, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 128)         147584    
_________________________________________________________________
max_

In [6]:
# If you want to train the same model or try other models, go for this

filepath = os.path.join("./emotion_detector_models/model_v6_23Aug_{epoch}.hdf5")

checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                             monitor='val_acc',
                                             verbose=1,
                                             save_best_only=True,
                                             mode='max')
callbacks = [checkpoint]
# if mode == "train":
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.003, decay=1e-6),metrics=['accuracy'])
nb_train_samples = train_generator.samples
nb_validation_samples = validation_generator.samples
epochs = 300
model_info = model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples // batch_size,
            epochs=epochs,
            callbacks = callbacks,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples // batch_size)

#plot_model_history(model_info)
#model.save_weights('model.h5')

Instructions for updating:
Use tf.cast instead.
Epoch 1/300
10/18 [===============>..............] - ETA: 7s - loss: 1.8132 - acc: 0.1922

C:\ProgramData\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 12 bytes but only got 11. Skipping tag 33432
  " Skipping tag %s" % (size, len(data), tag))


18/18 [==============================] - 16s 878ms/step - loss: 1.8053 - acc: 0.2005 - val_loss: 1.8390 - val_acc: 0.0729

Epoch 00001: val_acc improved from -inf to 0.07292, saving model to ./emotion_detector_models/model_v6_23Aug_1.hdf5
Epoch 2/300
18/18 [==============================] - 13s 711ms/step - loss: 1.7794 - acc: 0.2209 - val_loss: 1.7446 - val_acc: 0.2649

Epoch 00002: val_acc improved from 0.07292 to 0.26486, saving model to ./emotion_detector_models/model_v6_23Aug_2.hdf5
Epoch 3/300
18/18 [==============================] - 13s 697ms/step - loss: 1.7645 - acc: 0.2265 - val_loss: 1.6360 - val_acc: 0.2973

Epoch 00003: val_acc improved from 0.26486 to 0.29730, saving model to ./emotion_detector_models/model_v6_23Aug_3.hdf5
Epoch 4/300
18/18 [==============================] - 13s 699ms/step - loss: 1.7601 - acc: 0.2124 - val_loss: 1.7233 - val_acc: 0.3027

Epoch 00004: val_acc improved from 0.29730 to 0.30270, saving model to ./emotion_detector_models/model_v6_23Aug_4.hdf5

18/18 [==============================] - 11s 601ms/step - loss: 1.2327 - acc: 0.4804 - val_loss: 1.4169 - val_acc: 0.3892

Epoch 00043: val_acc did not improve from 0.44865
Epoch 44/300
18/18 [==============================] - 11s 596ms/step - loss: 1.2596 - acc: 0.4586 - val_loss: 1.5319 - val_acc: 0.3297

Epoch 00044: val_acc did not improve from 0.44865
Epoch 45/300
18/18 [==============================] - 11s 593ms/step - loss: 1.2303 - acc: 0.4789 - val_loss: 1.5416 - val_acc: 0.3333

Epoch 00045: val_acc did not improve from 0.44865
Epoch 46/300
18/18 [==============================] - 11s 602ms/step - loss: 1.2233 - acc: 0.4866 - val_loss: 1.4925 - val_acc: 0.3892

Epoch 00046: val_acc did not improve from 0.44865
Epoch 47/300
18/18 [==============================] - 11s 598ms/step - loss: 1.2287 - acc: 0.4875 - val_loss: 1.5384 - val_acc: 0.3351

Epoch 00047: val_acc did not improve from 0.44865
Epoch 48/300
18/18 [==============================] - 11s 609ms/step - loss: 1.2405

18/18 [==============================] - 11s 599ms/step - loss: 1.0067 - acc: 0.6059 - val_loss: 1.6720 - val_acc: 0.3892

Epoch 00086: val_acc did not improve from 0.49189
Epoch 87/300
18/18 [==============================] - 11s 604ms/step - loss: 1.0731 - acc: 0.5963 - val_loss: 1.4519 - val_acc: 0.4811

Epoch 00087: val_acc did not improve from 0.49189
Epoch 88/300
18/18 [==============================] - 11s 604ms/step - loss: 0.9963 - acc: 0.6087 - val_loss: 1.4018 - val_acc: 0.4865

Epoch 00088: val_acc did not improve from 0.49189
Epoch 89/300
18/18 [==============================] - 11s 621ms/step - loss: 1.0333 - acc: 0.6010 - val_loss: 1.3444 - val_acc: 0.4948

Epoch 00089: val_acc improved from 0.49189 to 0.49479, saving model to ./emotion_detector_models/model_v6_23Aug_89.hdf5
Epoch 90/300
18/18 [==============================] - 11s 627ms/step - loss: 0.9894 - acc: 0.6109 - val_loss: 1.3512 - val_acc: 0.5514

Epoch 00090: val_acc improved from 0.49479 to 0.55135, saving m

18/18 [==============================] - 11s 601ms/step - loss: 0.8501 - acc: 0.6792 - val_loss: 1.2899 - val_acc: 0.5833

Epoch 00129: val_acc did not improve from 0.63243
Epoch 130/300
18/18 [==============================] - 11s 604ms/step - loss: 0.9037 - acc: 0.6400 - val_loss: 1.2102 - val_acc: 0.5405

Epoch 00130: val_acc did not improve from 0.63243
Epoch 131/300
18/18 [==============================] - 11s 601ms/step - loss: 0.8851 - acc: 0.6612 - val_loss: 1.2168 - val_acc: 0.6541

Epoch 00131: val_acc improved from 0.63243 to 0.65405, saving model to ./emotion_detector_models/model_v6_23Aug_131.hdf5
Epoch 132/300
18/18 [==============================] - 11s 610ms/step - loss: 0.8566 - acc: 0.6814 - val_loss: 1.4412 - val_acc: 0.4919

Epoch 00132: val_acc did not improve from 0.65405
Epoch 133/300
18/18 [==============================] - 11s 599ms/step - loss: 0.8792 - acc: 0.6704 - val_loss: 1.1944 - val_acc: 0.5990

Epoch 00133: val_acc did not improve from 0.65405
Epoch 13


Epoch 00172: val_acc did not improve from 0.65405
Epoch 173/300
18/18 [==============================] - 11s 619ms/step - loss: 0.7766 - acc: 0.7078 - val_loss: 1.3931 - val_acc: 0.5156

Epoch 00173: val_acc did not improve from 0.65405
Epoch 174/300
18/18 [==============================] - 11s 597ms/step - loss: 0.8015 - acc: 0.6950 - val_loss: 1.5976 - val_acc: 0.4432

Epoch 00174: val_acc did not improve from 0.65405
Epoch 175/300
18/18 [==============================] - 11s 606ms/step - loss: 0.7925 - acc: 0.7005 - val_loss: 1.2354 - val_acc: 0.5189

Epoch 00175: val_acc did not improve from 0.65405
Epoch 176/300
18/18 [==============================] - 11s 604ms/step - loss: 0.7659 - acc: 0.7129 - val_loss: 1.3467 - val_acc: 0.5351

Epoch 00176: val_acc did not improve from 0.65405
Epoch 177/300
18/18 [==============================] - 11s 604ms/step - loss: 0.8540 - acc: 0.6759 - val_loss: 1.4175 - val_acc: 0.5156

Epoch 00177: val_acc did not improve from 0.65405
Epoch 178/300



Epoch 00216: val_acc did not improve from 0.65405
Epoch 217/300
18/18 [==============================] - 11s 603ms/step - loss: 0.7854 - acc: 0.7037 - val_loss: 1.2810 - val_acc: 0.5365

Epoch 00217: val_acc did not improve from 0.65405
Epoch 218/300
18/18 [==============================] - 11s 599ms/step - loss: 0.7569 - acc: 0.7194 - val_loss: 1.5729 - val_acc: 0.4541

Epoch 00218: val_acc did not improve from 0.65405
Epoch 219/300
18/18 [==============================] - 11s 600ms/step - loss: 0.7482 - acc: 0.7167 - val_loss: 1.3961 - val_acc: 0.5568

Epoch 00219: val_acc did not improve from 0.65405
Epoch 220/300
18/18 [==============================] - 11s 601ms/step - loss: 0.7464 - acc: 0.7208 - val_loss: 1.3069 - val_acc: 0.5405

Epoch 00220: val_acc did not improve from 0.65405
Epoch 221/300
18/18 [==============================] - 11s 597ms/step - loss: 0.7342 - acc: 0.7170 - val_loss: 1.5086 - val_acc: 0.4844

Epoch 00221: val_acc did not improve from 0.65405
Epoch 222/300


18/18 [==============================] - 11s 593ms/step - loss: 0.7028 - acc: 0.7349 - val_loss: 1.4204 - val_acc: 0.5892

Epoch 00260: val_acc did not improve from 0.70270
Epoch 261/300
18/18 [==============================] - 11s 599ms/step - loss: 0.7118 - acc: 0.7367 - val_loss: 1.3382 - val_acc: 0.6042

Epoch 00261: val_acc did not improve from 0.70270
Epoch 262/300
18/18 [==============================] - 11s 605ms/step - loss: 0.6665 - acc: 0.7581 - val_loss: 1.5401 - val_acc: 0.5676

Epoch 00262: val_acc did not improve from 0.70270
Epoch 263/300
18/18 [==============================] - 11s 599ms/step - loss: 0.7205 - acc: 0.7302 - val_loss: 1.3286 - val_acc: 0.6216

Epoch 00263: val_acc did not improve from 0.70270
Epoch 264/300
18/18 [==============================] - 11s 603ms/step - loss: 0.7267 - acc: 0.7260 - val_loss: 1.2006 - val_acc: 0.6216

Epoch 00264: val_acc did not improve from 0.70270
Epoch 265/300
18/18 [==============================] - 11s 598ms/step - loss: 0

In [ ]:
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [ ]:
plot_model_history(model_info)
model.save_weights('model_23Aug_724per.h5')

In [ ]:
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
# Found 28709 images belonging to 7 classes.
# Found 3589 images belonging to 7 classes.


# nb_train_samples = 28273
# nb_validation_samples = 3534
nb_train_samples = train_generator.samples
nb_validation_samples = validation_generator.samples

# We need to recreate our validation generator with shuffle = false
validation_generator = val_datagen.flow_from_directory(
        validation_data_dir,
        color_mode = 'grayscale',
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())

#Confution Matrix and Classification Report
Y_pred = model.predict_generator(validation_generator, nb_validation_samples // batch_size+1)
# print("Y_Pred: ", Y_pred)
y_pred = np.argmax(Y_pred, axis=1).reshape(-1,1)
print(y_pred.shape)

print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

plt.figure(figsize=(8,8))
cnf_matrix = confusion_matrix(validation_generator.classes, y_pred)

plt.imshow(cnf_matrix, interpolation='nearest')
plt.colorbar()
tick_marks = np.arange(len(classes))
_ = plt.xticks(tick_marks, classes, rotation=90)
_ = plt.yticks(tick_marks, classes)

In [ ]:
validation_generator.samples

In [ ]:
Y_pred[0].shape

# Loading our save model

In [ ]:
from keras.models import load_model
classifier = load_model('./emotion_detector_models/model_v6_23Aug_191.hdf5')

## Let's test on some of validation images


In [ ]:
from keras.models import load_model
from keras.optimizers import RMSprop, SGD, Adam
from keras.preprocessing import image
import numpy as np
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
import re

def draw_test(name, pred, im, true_label):
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 160, 0, 0, 300 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, "predited - "+ pred, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.putText(expanded_image, "true - "+ true_label, (20, 120) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,255,0), 2)
    cv2.imshow(name, expanded_image)


def getRandomImage(path, img_width, img_height):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    final_path = file_path + "/" + image_name
    return image.load_img(final_path, target_size = (img_width, img_height),grayscale=True), final_path, path_class

# dimensions of our images
img_width, img_height = 48, 48

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

files = []
predictions = []
true_labels = []

# predicting images
for i in range(0, 50):
    path = '.\Dataset\\Validation_dataset\\' 
    img, final_path, true_label = getRandomImage(path, img_width, img_height)
    files.append(final_path)
    true_labels.append(true_label)
    x = image.img_to_array(img)
    x = x * 1./255
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    classes = model.predict_classes(images, batch_size = 10)
    predictions.append(classes)
    
for i in range(0, len(files)):
    image = cv2.imread((files[i]))
    image = cv2.resize(image, None, fx=1, fy=1, interpolation = cv2.INTER_CUBIC)
    draw_test("Prediction", class_labels[predictions[i][0]], image, true_labels[i])
    cv2.waitKey(0)

cv2.destroyAllWindows()

# Test on a single image

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
from keras.preprocessing.image import img_to_array

face_classifier = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

def face_detector(img):
    # Convert image to grayscale
    gray = cv2.cvtColor(img.copy(),cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return (0,0,0,0), np.zeros((48,48), np.uint8), img
    
    allfaces = []   
    rects = []
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation = cv2.INTER_AREA)
        allfaces.append(roi_gray)
        rects.append((x,w,y,h))
    return rects, allfaces, img

img_path = r"C:\Users\naggrawa\Downloads\Nishi\Facial Expression\Program\Dataset\Test Dataset\0fe58a7772a9d6ad847c5c3ae8f8412e.jpg"
img = cv2.imread(img_path)
rects, faces, image = face_detector(img)

i = 0
for face in faces:
    roi = face.astype("float") / 255.0
    roi = img_to_array(roi)
    roi = np.expand_dims(roi, axis=0)

    # make a prediction on the ROI, then lookup the class
    preds = classifier.predict(roi)[0]
    label = class_labels[preds.argmax()]   

    #Overlay our detected emotion on our pic
    label_position = (rects[i][0] + int((rects[i][1]/2)), abs(rects[i][2] - 10))
    i =+ 1
    cv2.putText(image, label, label_position , cv2.FONT_HERSHEY_SIMPLEX,1, (0,255,0), 2)
    
cv2.imshow("Emotion Detector", image)
cv2.waitKey(0)

cv2.destroyAllWindows()